In [ ]:
import os
import json

import numpy as np
import pandas as pd
import seaborn as sn
import matplotlib.pyplot as plt
import cv2
import albumentations as A


import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('fivethirtyeight')
sns.set_style('whitegrid')
BASE_DIR = "./cassava-leaf-disease-classification/"

In [ ]:
with open(os.path.join(BASE_DIR, "label_num_to_disease_map.json")) as file:
    map_classes = json.loads(file.read())
    map_classes = {int(k) : v for k, v in map_classes.items()}
    
print(json.dumps(map_classes, indent=4))

In [ ]:
input_files = os.listdir(os.path.join(BASE_DIR, "train_images"))
print(f"Number of images: {len(input_files)}")

In [ ]:
import cv2
img_shapes = {}
for image_name in os.listdir(os.path.join(BASE_DIR, "train_images"))[:10]:
    image = cv2.imread(os.path.join(BASE_DIR, "train_images", image_name))
    img_shapes[image.shape] = img_shapes.get(image.shape, 0) + 1

print(img_shapes)

In [ ]:
import pandas as pd
df_train = pd.read_csv(os.path.join(BASE_DIR, "train.csv"))

df_train["class_name"] = df_train["label"].map(map_classes)

df_train

distribution of classes image

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sn

plt.figure(figsize=(8, 4))
sn.countplot(y="class_name", data=df_train)

display how the different classes look like


In [ ]:
from matplotlib.ticker import MaxNLocator
import random

def display_samples(df, num_samples=9, display_first=5):
    plt.figure(figsize=(30, 30))
    
    # Iterate through unique class labels
    for label in df['label'].unique():
        # Get the first 30 samples for the current class
        class_samples = df[df['label'] == label].head(30)
        
        # Randomly select 5 samples from the first 30
        selected_samples = class_samples.sample(min(display_first, len(class_samples)), random_state=42)
        
        # Iterate through selected samples of the current class
        for i, row in enumerate(selected_samples.itertuples(), 1):
            img_path = os.path.join(BASE_DIR, "train_images", row.image_id)
            img = plt.imread(img_path)

            # Calculate subplot index
            subplot_index = (label * num_samples) + i
            
            # Create subplot
            plt.subplot(len(df['label'].unique()), num_samples, subplot_index)
            plt.imshow(img)
            
            # Display class label as the title
            if i == 1:  # Display the title only once for each class
                plt.title(f'Class {row.label}')
                
            # Turn off the axis for each subplot
            plt.axis('off')

    plt.subplots_adjust(wspace=0.1, hspace=0.1)  # Adjust the spacing between subplots
    plt.show()

# Call the function with your DataFrame
display_samples(df_train, num_samples=5, display_first=5)


Display how the images will look before and after resizing, greyscale and sharpened

In [ ]:
from matplotlib.ticker import MaxNLocator
import random

def display_samples(df, num_samples=9, display_first=30):
    plt.figure(figsize=(30, 30))
    
    # Iterate through unique class labels
    for label in df['label'].unique():
        # Get the first 30 samples for the current class
        class_samples = df[df['label'] == label].head(30)
        
        # Randomly select 5 samples from the first 30
        selected_samples = class_samples.sample(min(display_first, len(class_samples)), random_state=42)
    
    # Return the selected samples for further use
    return selected_samples

# Call the function with your DataFrame
selected_samples = display_samples(df_train, num_samples=5, display_first=30)

# Function to sharpen the image
def sharpen_image(image):
    image_blurred = cv2.GaussianBlur(image, (0, 0), 3)
    image_sharp = cv2.addWeighted(image, 1.5, image_blurred, -0.5, 0)
    return image_sharp

# Function to convert the image to grayscale
def gray_image(image):
    output = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    return output / 255

# Pick 3 random images
random_samples = selected_samples.sample(3, random_state=42)

# Display the original, resized, grayscale, and sharpened images
for _, row in random_samples.iterrows():
    img_path = os.path.join(BASE_DIR, "train_images", row.image_id)
    img = plt.imread(img_path)

    # Resize the image
    resized_img = cv2.resize(img, (150, 150))

    # Convert to grayscale
    gray_img = gray_image(resized_img)

    # Sharpen the image
    sharpened_img = sharpen_image(resized_img)

    # Plot the images
    plt.figure(figsize=(12, 4))
    
    plt.subplot(1, 4, 1)
    plt.imshow(img)
    plt.title("Original")
    
    plt.subplot(1, 4, 2)
    plt.imshow(resized_img)
    plt.title("Resized (150x150)")
    
    plt.subplot(1, 4, 3)
    plt.imshow(gray_img, cmap='gray')
    plt.title("Grayscale")
    
    plt.subplot(1, 4, 4)
    plt.imshow(sharpened_img)
    plt.title("Sharpened")

    plt.show()
